In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import LongType,DateType,StringType
from pyspark.sql.types import StructField, StructType, BooleanType, ArrayType, IntegerType, DoubleType, FloatType
from delta import *
from delta.tables import *
from hdfs import InsecureClient
from os import PathLike
import pandas as teste
from pyspark.sql.functions import *
from pyspark.sql.functions import max as sparkMax
from pyspark.sql.functions import substring, avg, sum

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/project/uc2/silver/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration Porject") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# create gold database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS UC2gold LOCATION 'hdfs://hdfs-nn:9000/project/uc2/gold/warehouse/UC2gold.db/'
    """
)
# create table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS UC2gold.gold_vehicle_information
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE UC2gold.gold_vehicle_information (
        Accident_Index STRING,
        Age_Band_of_Driver STRING,
        Age_of_Vehicle INTEGER,
        Engine_Capacity_CC STRING,
        Brand STRING,
        Vehicle_Type STRING,
        Accident_year INT,
        Accident_Severity STRING,
        Day_of_Week STRING,
        Number_of_Casualties INT
        )
        
    LOCATION 'hdfs://hdfs-nn:9000/project/uc2/gold/warehouse/UC2gold.db/gold_vehicle_information/'
    """
)


DataFrame[]

In [3]:
# read data from the silver tables
df_final_Cris = spark.table("vehicle_silver.vehicle_silver_deltalake_table")
df_final_Sara = spark.table("Accident_silver.deltalake_table")

# Join
join_df = df_final_Cris \
    .join(df_final_Sara, df_final_Cris.Accident_Index == df_final_Sara.Accident_Index) \
    .select("vehicle_silver_deltalake_table.Accident_Index", "vehicle_silver_deltalake_table.Age_Band_of_Driver","vehicle_silver_deltalake_table.Age_of_Vehicle","vehicle_silver_deltalake_table.Engine_Capacity_CC","vehicle_silver_deltalake_table.Brand","vehicle_silver_deltalake_table.Vehicle_Type","vehicle_silver_deltalake_table.Accident_year","Accident_Severity","Day_of_Week","Number_of_Casualties") \

join_df.show()


+--------------+--------------------+--------------+------------------+----------------+--------------------+-------------+-----------------+-----------+--------------------+
|Accident_Index|  Age_Band_of_Driver|Age_of_Vehicle|Engine_Capacity_CC|           Brand|        Vehicle_Type|Accident_year|Accident_Severity|Day_of_Week|Number_of_Casualties|
+--------------+--------------------+--------------+------------------+----------------+--------------------+-------------+-----------------+-----------+--------------------+
| 200501BS70192|             26 - 35|             8|              1896|      VOLKSWAGEN|Van / Goods 3.5 t...|         2005|           Slight|  Wednesday|                   1|
| 200501BS70739|Data missing or o...|             3|              1598|          TOYOTA|                 Car|         2005|           Slight|   Saturday|                   1|
| 200501CP00182|Data missing or o...|             3|              1896|      VOLKSWAGEN|                 Car|         2005|  

In [4]:
# Aggregate
gold_per_brand_df = join_df \
    .groupBy("vehicle_silver_deltalake_table.Accident_year","vehicle_silver_deltalake_table.Engine_Capacity_CC","vehicle_silver_deltalake_table.Brand") \
    .agg(
        count(join_df.Accident_Index).cast('STRING').alias("Accident_Index"),
        #sum(join_df.Accident_Index).cast('STRING').alias("Accident_Index")
    ) \
    .orderBy("Accident_year")
    

# write to delta table
gold_per_brand_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true")\
    .save("hdfs://hdfs-nn:9000/project/uc2/gold/warehouse/UC2gold.db/gold_vehicle_information/gold_per_brand_df/")
  
gold_per_brand_df.count()

35793

In [5]:
# check the results in the table
gold_per_brand_df.toPandas()

,Accident_year,Engine_Capacity_CC,Brand,Accident_Index
0,2005,1165,KAWASAKI,14
1,2005,1200,BUELL,14
2,2005,1270,NISSAN,19
3,2005,2988,NISSAN,2
4,2005,248,KAWASAKI,7
...,...,...,...,...
35788,2016,10000,FODEN,1
35789,2016,847,PERODUA,2
35790,2016,1002,BIMOTA,1
35791,2016,645,BMW,1


In [6]:
gold_per_brand_df.show(1000)

+-------------+------------------+------------------+--------------+
|Accident_year|Engine_Capacity_CC|             Brand|Accident_Index|
+-------------+------------------+------------------+--------------+
|         2005|              2460|             VOLVO|             3|
|         2005|              1339|             HONDA|           347|
|         2005|              3000|               BMW|            18|
|         2005|              1100|             ROVER|             2|
|         2005|              8000|   SEDDON/ATKINSON|             2|
|         2005|               998|            AUSTIN|            52|
|         2005|              1781|              SEAT|            67|
|         2005|              1047|             HONDA|             1|
|         2005|              1398|           RENAULT|             1|
|         2005|              1900|           PEUGEOT|             2|
|         2005|              2148|          CHRYSLER|             8|
|         2005|              6224|

In [7]:
spark.stop()